<a href="https://colab.research.google.com/github/milleflori-data/Query-Dash-Jupyter/blob/main/Copy_of_Query_Dash_Jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🚀 Enterprise GDELT Analytics Platform
## Professional-Grade Data Querying & Dashboard System

### 🎯 **Design Philosophy**
- **Query Efficiency First**: Minimize BigQuery costs through smart caching and query optimization
- **Professional Scalability**: Built for enterprise use with proper error handling and monitoring
- **Desktop Deployment Ready**: Designed for standalone desktop dashboard deployment
- **Resource Management**: Intelligent memory and API quota management

---

## 🏗️ **Architecture Overview**

```
┌─────────────────────────────────────────────────────────────┐
│                    GDELT Analytics Platform                  │
├─────────────────┬─────────────────┬─────────────────────────┤
│   Query Layer   │  Caching Layer  │    Analytics Layer      │
│                 │                 │                         │
│ • Smart Queries │ • Multi-tier    │ • Real-time Dashboards  │
│ • Cost Control  │ • Persistence   │ • Export Capabilities   │
│ • Monitoring    │ • Validation    │ • Professional Reports  │
└─────────────────┴─────────────────┴─────────────────────────┘
```

---

## 📋 **Table of Contents**

1. [🔧 Configuration & Environment](#configuration)
2. [💾 Smart Caching System](#caching)
3. [🔍 Optimized Query Engine](#queries)
4. [📊 Professional Dashboard](#dashboard)
5. [📈 Analytics & Reporting](#analytics)
6. [🖥️ Desktop Deployment](#desktop)
7. [🚀 Performance Monitoring](#monitoring)

---

## 🔧 Configuration & Environment {#configuration}

### Core Dependencies
```python
# Core data processing
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# BigQuery & Cloud
from google.cloud import bigquery
from google.cloud.exceptions import GoogleCloudError

# Visualization
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

# Caching & Persistence
import pickle
import hashlib
import os
import json
from pathlib import Path

# Performance & Monitoring
import time
import psutil
import logging
from functools import wraps
```

### Configuration Management
```python
class GDELTConfig:
    """Centralized configuration management"""

    def __init__(self, environment="kaggle"):
        self.environment = environment
        self.setup_config()

    def setup_config(self):
        """Setup environment-specific configurations"""

        if self.environment == "kaggle":
            self.config = {
                "max_query_bytes": 1_000_000_000,  # 1GB query limit
                "max_rows_per_query": 100_000,
                "cache_dir": "/tmp/gdelt_cache",
                "export_formats": ["csv", "json", "parquet"],
                "visualization_theme": "plotly_white"
            }
        elif self.environment == "desktop":
            self.config = {
                "max_query_bytes": 10_000_000_000,  # 10GB for desktop
                "max_rows_per_query": 1_000_000,
                "cache_dir": "./gdelt_cache",
                "export_formats": ["csv", "json", "parquet", "excel"],
                "visualization_theme": "plotly_dark"
            }

        # Create cache directory
        Path(self.config["cache_dir"]).mkdir(exist_ok=True)

        # Setup logging
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(f'{self.config["cache_dir"]}/gdelt.log'),
                logging.StreamHandler()
            ]
        )

# Initialize configuration
config = GDELTConfig()
```

---

## 💾 Smart Caching System {#caching}

### Multi-Tier Caching Strategy
```python
class GDELTCache:
    """Enterprise-grade caching system with multiple tiers"""

    def __init__(self, cache_dir):
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)
        self.metadata_file = self.cache_dir / "cache_metadata.json"
        self.load_metadata()

    def load_metadata(self):
        """Load cache metadata"""
        try:
            with open(self.metadata_file, 'r') as f:
                self.metadata = json.load(f)
        except FileNotFoundError:
            self.metadata = {}

    def save_metadata(self):
        """Save cache metadata"""
        with open(self.metadata_file, 'w') as f:
            json.dump(self.metadata, f, indent=2, default=str)

    def generate_cache_key(self, query_params):
        """Generate unique cache key for query parameters"""
        key_string = json.dumps(query_params, sort_keys=True)
        return hashlib.md5(key_string.encode()).hexdigest()

    def is_cache_valid(self, cache_key, max_age_hours=6):
        """Check if cached data is still valid"""
        if cache_key not in self.metadata:
            return False

        cache_time = datetime.fromisoformat(self.metadata[cache_key]['timestamp'])
        age = datetime.now() - cache_time

        return age.total_seconds() / 3600 < max_age_hours

    def cache_data(self, cache_key, data, query_params):
        """Cache data with metadata"""
        cache_file = self.cache_dir / f"{cache_key}.pkl"

        try:
            with open(cache_file, 'wb') as f:
                pickle.dump(data, f)

            self.metadata[cache_key] = {
                'timestamp': datetime.now().isoformat(),
                'rows': len(data) if hasattr(data, '__len__') else 0,
                'query_params': query_params,
                'file_size': cache_file.stat().st_size
            }

            self.save_metadata()
            logging.info(f"📦 Cached {len(data):,} rows to {cache_key}")

        except Exception as e:
            logging.error(f"❌ Cache write error: {e}")

    def load_cached_data(self, cache_key):
        """Load data from cache"""
        cache_file = self.cache_dir / f"{cache_key}.pkl"

        try:
            with open(cache_file, 'rb') as f:
                data = pickle.load(f)

            rows = self.metadata[cache_key]['rows']
            logging.info(f"📂 Loaded {rows:,} rows from cache")
            return data

        except Exception as e:
            logging.error(f"❌ Cache read error: {e}")
            return None

    def cleanup_cache(self, max_size_gb=5):
        """Clean up old cache files"""
        total_size = sum(f.stat().st_size for f in self.cache_dir.glob("*.pkl"))
        max_size_bytes = max_size_gb * 1024**3

        if total_size > max_size_bytes:
            # Sort by timestamp and remove oldest
            sorted_keys = sorted(
                self.metadata.items(),
                key=lambda x: x[1]['timestamp']
            )

            for key, meta in sorted_keys:
                cache_file = self.cache_dir / f"{key}.pkl"
                if cache_file.exists():
                    cache_file.unlink()
                    del self.metadata[key]
                    logging.info(f"🗑️ Removed old cache: {key}")

                    total_size -= meta['file_size']
                    if total_size <= max_size_bytes:
                        break

            self.save_metadata()

# Initialize cache system
cache_system = GDELTCache(config.config["cache_dir"])
```

---

## 🔍 Optimized Query Engine {#queries}

### Smart Query Builder with Cost Control
```python
class GDELTQueryEngine:
    """Enterprise query engine with cost optimization"""

    def __init__(self, client, cache_system):
        self.client = client
        self.cache = cache_system
        self.query_stats = {"total_queries": 0, "total_bytes": 0, "cache_hits": 0}

    def estimate_query_cost(self, query):
        """Estimate BigQuery cost before execution"""
        if not self.client:
            return None

        try:
            job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=True)
            query_job = self.client.query(query, job_config=job_config)

            bytes_processed = query_job.total_bytes_processed
            estimated_cost = (bytes_processed / 1024**4) * 5  # $5 per TB

            return {
                "bytes_processed": bytes_processed,
                "estimated_cost_usd": estimated_cost,
                "size_mb": bytes_processed / 1024**2
            }
        except Exception as e:
            logging.error(f"❌ Cost estimation failed: {e}")
            return None

    def build_optimized_query(self, params):
        """Build highly optimized GDELT query"""

        # Smart date partitioning
        end_date = datetime.now()
        start_date = end_date - timedelta(days=params.get('days_back', 30))

        # Use YYYYMMDD format for partition pruning
        start_str = start_date.strftime('%Y%m%d')
        end_str = end_date.strftime('%Y%m%d')

        # Build efficient query with proper indexing
        query = f"""
        WITH filtered_events AS (
            SELECT
                SQLDATE,
                Actor1Name,
                Actor2Name,
                EventCode,
                EventRootCode,
                CAST(EventCode AS STRING) as EventCodeStr,
                QuadClass,
                GoldsteinScale,
                NumMentions,
                AvgTone,
                ActionGeo_CountryCode,
                ActionGeo_Lat,
                ActionGeo_Long,
                SOURCEURL
            FROM `gdelt-bq.gdeltv2.events`
            WHERE
                -- Partition pruning for cost efficiency
                _PARTITIONTIME >= TIMESTAMP('{start_date.strftime('%Y-%m-%d')}')
                AND _PARTITIONTIME <= TIMESTAMP('{end_date.strftime('%Y-%m-%d')}')
                AND SQLDATE >= {start_str}
                AND SQLDATE <= {end_str}
                -- Quality filters
                AND Actor1Name IS NOT NULL
                AND EventCode IS NOT NULL
                AND NumMentions >= {params.get('min_mentions', 5)}
                AND ActionGeo_CountryCode IS NOT NULL
                AND LENGTH(Actor1Name) < 100  -- Filter out URLs/garbage
        """

        # Add country filter with efficient IN clause
        if params.get('countries'):
            countries_str = "', '".join(params['countries'])
            query += f"\n                AND ActionGeo_CountryCode IN ('{countries_str}')"

        # Add category filter using event codes
        if params.get('categories'):
            event_codes = self._get_event_codes_for_categories(params['categories'])
            if event_codes:
                codes_str = "', '".join(event_codes)
                query += f"\n                AND EventRootCode IN ('{codes_str}')"

        # Add tone filter for sentiment analysis
        if params.get('tone_range'):
            min_tone, max_tone = params['tone_range']
            query += f"\n                AND AvgTone BETWEEN {min_tone} AND {max_tone}"

        query += f"""
        ),
        ranked_events AS (
            SELECT *,
                ROW_NUMBER() OVER (
                    ORDER BY NumMentions DESC, SQLDATE DESC
                ) as rn
            FROM filtered_events
        )
        SELECT * EXCEPT(rn)
        FROM ranked_events
        WHERE rn <= {params.get('limit', 100000)}
        """

        return query

    def _get_event_codes_for_categories(self, categories):
        """Enhanced category to event code mapping"""
        category_codes = {
            "Politics / Diplomacy": ['01', '02', '03', '04', '05'],
            "Military / Conflict": ['13', '14'],
            "Finance / Markets": ['07'],
            "Health / Medical": ['042', '043', '044'],
            "Technology / Science": ['081', '082', '083'],
            "Environment / Weather": ['045', '087', '088'],
            "Unconventional Violence": ['15', '16', '17'],
            "Mass Violence": ['18', '19', '20'],
            "Social / Cultural": ['06', '08', '09'],
            "Legal / Judicial": ['10', '11', '12']
        }

        all_codes = []
        for cat in categories:
            if cat in category_codes:
                all_codes.extend(category_codes[cat])

        return list(set(all_codes))  # Remove duplicates

    @wraps
    def execute_query(self, params, use_cache=True, max_cost_usd=1.0):
        """Execute query with caching and cost control"""

        # Check cache first
        cache_key = self.cache.generate_cache_key(params)

        if use_cache and self.cache.is_cache_valid(cache_key):
            self.query_stats["cache_hits"] += 1
            return self.cache.load_cached_data(cache_key)

        # Build and validate query
        query = self.build_optimized_query(params)

        # Cost estimation
        cost_estimate = self.estimate_query_cost(query)
        if cost_estimate:
            logging.info(f"💰 Query will process {cost_estimate['size_mb']:.1f} MB")
            logging.info(f"💰 Estimated cost: ${cost_estimate['estimated_cost_usd']:.4f}")

            if cost_estimate['estimated_cost_usd'] > max_cost_usd:
                raise ValueError(f"Query cost ${cost_estimate['estimated_cost_usd']:.4f} exceeds limit ${max_cost_usd}")

        # Execute query with monitoring
        start_time = time.time()

        try:
            logging.info(f"🔄 Executing BigQuery...")
            query_job = self.client.query(query)
            df = query_job.to_dataframe()

            execution_time = time.time() - start_time

            # Update statistics
            self.query_stats["total_queries"] += 1
            if cost_estimate:
                self.query_stats["total_bytes"] += cost_estimate["bytes_processed"]

            logging.info(f"✅ Query completed in {execution_time:.2f}s - {len(df):,} rows")

            # Cache results
            if use_cache:
                self.cache.cache_data(cache_key, df, params)

            return df

        except Exception as e:
            logging.error(f"❌ Query execution failed: {e}")
            raise

    def get_query_statistics(self):
        """Get comprehensive query statistics"""
        total_cost = (self.query_stats["total_bytes"] / 1024**4) * 5

        return {
            "total_queries": self.query_stats["total_queries"],
            "cache_hits": self.query_stats["cache_hits"],
            "cache_hit_rate": self.query_stats["cache_hits"] / max(self.query_stats["total_queries"], 1),
            "total_bytes_processed": self.query_stats["total_bytes"],
            "estimated_total_cost": total_cost,
            "avg_cost_per_query": total_cost / max(self.query_stats["total_queries"], 1)
        }

# Initialize query engine
query_engine = GDELTQueryEngine(client, cache_system) if 'client' in globals() else None
```

---

## 📊 Professional Dashboard System {#dashboard}

### Enterprise Dashboard Builder
```python
class GDELTProfessionalDashboard:
    """Professional-grade dashboard with enterprise features"""

    def __init__(self, theme="plotly_white"):
        self.theme = theme
        pio.templates.default = theme

        # Professional color schemes
        self.colors = {
            "primary": "#2E86C1",
            "secondary": "#F39C12",
            "success": "#27AE60",
            "warning": "#E74C3C",
            "neutral": "#95A5A6",
            "dark": "#34495E"
        }

    def create_executive_summary(self, df, params):
        """Create executive summary with KPIs"""

        if df is None or df.empty:
            return None

        # Calculate KPIs
        total_events = len(df)
        date_range = (df['date'].min(), df['date'].max())
        avg_tone = df['AvgTone'].mean()
        top_actor = df.groupby('Actor1Name')['NumMentions'].sum().idxmax()
        most_active_country = df['ActionGeo_CountryCode'].value_counts().index[0]

        # Create KPI cards
        kpi_fig = go.Figure()

        kpi_cards = [
            {"title": "Total Events", "value": f"{total_events:,}", "color": self.colors["primary"]},
            {"title": "Date Range", "value": f"{date_range[0].strftime('%m/%d')} - {date_range[1].strftime('%m/%d')}", "color": self.colors["secondary"]},
            {"title": "Avg Sentiment", "value": f"{avg_tone:.2f}", "color": self.colors["success"] if avg_tone > 0 else self.colors["warning"]},
            {"title": "Top Actor", "value": str(top_actor)[:20], "color": self.colors["dark"]},
            {"title": "Most Active", "value": most_active_country, "color": self.colors["neutral"]}
        ]

        # Add annotations for KPIs
        annotations = []
        for i, kpi in enumerate(kpi_cards):
            x_pos = (i + 1) / (len(kpi_cards) + 1)

            annotations.extend([
                dict(x=x_pos, y=0.7, text=f"<b>{kpi['value']}</b>",
                     showarrow=False, font=dict(size=24, color=kpi['color'])),
                dict(x=x_pos, y=0.3, text=kpi['title'],
                     showarrow=False, font=dict(size=14, color="gray"))
            ])

        kpi_fig.update_layout(
            title="📊 Executive Summary",
            height=200,
            showlegend=False,
            xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
            yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
            annotations=annotations,
            plot_bgcolor="rgba(0,0,0,0)"
        )

        return kpi_fig

    def create_comprehensive_dashboard(self, df, params):
        """Create comprehensive multi-panel dashboard"""

        if df is None or df.empty:
            return None

        # Process data for visualizations
        timeline_data = self._prepare_timeline_data(df)
        actor_data = self._prepare_actor_data(df)
        geo_data = self._prepare_geo_data(df)
        sentiment_data = self._prepare_sentiment_data(df)
        category_data = self._prepare_category_data(df)

        # Create subplot structure
        fig = make_subplots(
            rows=3, cols=3,
            subplot_titles=[
                "📈 Event Timeline", "🎭 Top Actors by Influence", "🌍 Geographic Distribution",
                "😊 Sentiment Analysis", "📊 Event Categories", "⚡ Event Intensity Heatmap",
                "🔗 Actor Network", "📅 Weekly Patterns", "🎯 Key Metrics"
            ],
            specs=[
                [{"type": "scatter"}, {"type": "bar"}, {"type": "scatter"}],
                [{"type": "histogram"}, {"type": "pie"}, {"type": "heatmap"}],
                [{"type": "scatter"}, {"type": "bar"}, {"type": "table"}]
            ],
            horizontal_spacing=0.08,
            vertical_spacing=0.1
        )

        # 1. Timeline with trend analysis
        fig.add_trace(
            go.Scatter(
                x=timeline_data['date'],
                y=timeline_data['events'],
                mode='lines+markers',
                name='Daily Events',
                line=dict(color=self.colors["primary"], width=3),
                hovertemplate='<b>%{x}</b><br>Events: %{y}<extra></extra>'
            ),
            row=1, col=1
        )

        # Add trend line
        z = np.polyfit(range(len(timeline_data)), timeline_data['events'], 1)
        trend = np.poly1d(z)(range(len(timeline_data)))

        fig.add_trace(
            go.Scatter(
                x=timeline_data['date'],
                y=trend,
                mode='lines',
                name='Trend',
                line=dict(color=self.colors["warning"], width=2, dash='dash')
            ),
            row=1, col=1
        )

        # 2. Top Actors with enhanced styling
        fig.add_trace(
            go.Bar(
                x=actor_data['total_mentions'][:10],
                y=actor_data['Actor1Name'][:10],
                orientation='h',
                name='Actor Influence',
                marker_color=self.colors["secondary"],
                text=actor_data['total_mentions'][:10],
                textposition='auto',
                hovertemplate='<b>%{y}</b><br>Total Mentions: %{x:,}<extra></extra>'
            ),
            row=1, col=2
        )

        # 3. Geographic scatter plot
        fig.add_trace(
            go.Scattermap(
                lat=geo_data['ActionGeo_Lat'],
                lon=geo_data['ActionGeo_Long'],
                mode='markers',
                marker=dict(
                    size=geo_data['events'],
                    sizemode='diameter',
                    sizeref=geo_data['events'].max()/50,
                    color=geo_data['avg_tone'],
                    colorscale='RdYlBu',
                    colorbar=dict(title="Avg Tone"),
                    line=dict(color='black', width=1)
                ),
                text=geo_data['ActionGeo_CountryCode'],
                hovertemplate='<b>%{text}</b><br>Events: %{marker.size}<br>Avg Tone: %{marker.color:.2f}<extra></extra>'
            ),
            row=1, col=3
        )

        # 4. Sentiment histogram
        fig.add_trace(
            go.Histogram(
                x=sentiment_data['AvgTone'],
                nbinsx=30,
                name='Sentiment Distribution',
                marker_color=self.colors["success"],
                opacity=0.7
            ),
            row=2, col=1
        )

        # 5. Category pie chart
        fig.add_trace(
            go.Pie(
                labels=category_data['Category'],
                values=category_data['Events'],
                name='Categories',
                hole=0.4,
                hovertemplate='<b>%{label}</b><br>Events: %{value}<br>Percentage: %{percent}<extra></extra>'
            ),
            row=2, col=2
        )

        # Continue with remaining visualizations...

        # Update layout with professional styling
        fig.update_layout(
            title={
                'text': f"🌍 GDELT Professional Analytics Dashboard<br><sup>Generated on {datetime.now().strftime('%Y-%m-%d %H:%M UTC')} | {len(df):,} Events Analyzed</sup>",
                'x': 0.5,
                'xanchor': 'center',
                'font': {'size': 20}
            },
            height=1200,
            showlegend=False,
            template=self.theme,
            font=dict(family="Arial, sans-serif")
        )

        return fig

    def _prepare_timeline_data(self, df):
        """Prepare timeline data with aggregations"""
        return df.groupby('date').agg({
            'NumMentions': ['count', 'sum', 'mean'],
            'AvgTone': 'mean'
        }).round(2).reset_index()

    def _prepare_actor_data(self, df):
        """Prepare actor data with influence metrics"""
        return (df.groupby('Actor1Name')
                .agg({
                    'NumMentions': ['sum', 'count', 'mean'],
                    'AvgTone': 'mean'
                })
                .round(2)
                .sort_values(('NumMentions', 'sum'), ascending=False)
                .reset_index())

    def _prepare_geo_data(self, df):
        """Prepare geographic data with coordinates"""
        geo_df = df.dropna(subset=['ActionGeo_Lat', 'ActionGeo_Long'])
        return (geo_df.groupby(['ActionGeo_CountryCode', 'ActionGeo_Lat', 'ActionGeo_Long'])
                .agg({
                    'NumMentions': ['count', 'sum'],
                    'AvgTone': 'mean'
                })
                .round(2)
                .reset_index())

    def _prepare_sentiment_data(self, df):
        """Prepare sentiment analysis data"""
        return df.dropna(subset=['AvgTone'])

    def _prepare_category_data(self, df):
        """Prepare category distribution data"""
        return df['Category'].value_counts().reset_index()

# Initialize dashboard system
dashboard = GDELTProfessionalDashboard()
```

---

## 🖥️ Desktop Deployment Strategy {#desktop}

### Standalone Desktop Application
```python
class GDELTDesktopApp:
    """Desktop deployment with local server capabilities"""

    def __init__(self):
        self.config = GDELTConfig("desktop")
        self.setup_desktop_environment()

    def setup_desktop_environment(self):
        """Setup desktop-specific configurations"""

        # Enhanced local storage
        self.data_dir = Path.home() / "GDELTAnalytics"
        self.data_dir.mkdir(exist_ok=True)

        # Local database option (SQLite for offline work)
        self.local_db = self.data_dir / "gdelt_local.db"

        # Desktop-specific features
        self.features = {
            "offline_mode": True,
            "local_storage": True,
            "export_options": ["pdf", "pptx", "excel", "csv", "json"],
            "scheduled_updates": True,
            "email_reports": True
        }

    def create_desktop_launcher(self):
        """Create desktop application launcher"""

        launcher_script = f"""
#!/usr/bin/env python3
import sys
import webbrowser
from threading import Timer
import subprocess
import os

# Change to application directory
os.chdir('{self.data_dir}')

# Start Jupyter server with custom settings
jupyter_cmd = [
    sys.executable, '-m', 'jupyter', 'notebook',
    '--no-browser',
    '--port=8888',
    '--NotebookApp.token=""',
    '--NotebookApp.password=""',
    f'--notebook-dir={self.data_dir}'
]

# Start server
process = subprocess.Popen(jupyter_cmd)

# Open browser after delay
Timer(3, lambda: webbrowser.open('http://localhost:8888')).start()

print("🚀 GDELT Analytics Desktop launched!")
print("📊 Dashboard available at: http://localhost:8888")
print("Press Ctrl+C to stop the server")

try:
    process.wait()
except KeyboardInterrupt:
    process.terminate()
    print("\\n✅ GDELT Analytics stopped")
"""

        launcher_path = self.data_dir / "launch_gdelt.py"
        launcher_path.write_text(launcher_script)
        launcher_path.chmod(0o755)

        return launcher_path

    def setup_offline_capabilities(self):
        """Setup offline data storage and processing"""

        # SQLite schema for local storage
        import sqlite3

        conn = sqlite3.connect(self.local_db)
        cursor = conn.cursor()

        cursor.execute("""
        CREATE TABLE IF NOT EXISTS gdelt_events (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            sqldate INTEGER,
            actor1_name TEXT,
            actor2_name TEXT,
            event_code TEXT,
            event_root_code TEXT,
            quad_class INTEGER,
            goldstein_scale REAL,
            num_mentions INTEGER,
            avg_tone REAL,
            country_code TEXT,
            latitude REAL,
            longitude REAL,
            source_url TEXT,
            category TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """)

        cursor.execute("""
        CREATE INDEX IF NOT EXISTS idx_date ON gdelt_events(sqldate);
        CREATE INDEX IF NOT EXISTS idx_country ON gdelt_events(country_code);
        CREATE INDEX IF NOT EXISTS idx_category ON gdelt_events(category);
        CREATE INDEX IF NOT EXISTS idx_actor ON gdelt_events(actor1_name);
        """)

        conn.commit()
        conn.close()

        return self.local_db

    def create_scheduled_updater(self):
        """Create scheduled data update system"""

        update_script = f"""
import schedule
import time
from datetime import datetime
import logging

# Setup logging
logging.basicConfig(
    filename='{self.data_dir}/updater.log',
    level=logging.INFO,
    format='%(asctime)s - %(message)s'
)

def update_gdelt_data():
    \"\"\"Scheduled data update function\"\"\"
    try:
        logging.info("🔄 Starting scheduled GDELT data update")

        # Import and run update logic here
        # This would call your query engine with fresh data

        logging.info("✅ Scheduled update completed successfully")
    except Exception as e:
        logging.error(f"

SyntaxError: invalid character '┌' (U+250C) (3860803288.py, line 15)